In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel,Field

In [5]:
model = ChatOpenAI(model_name = 'gpt-4o-mini')

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [8]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

In [9]:
prompt = prompt.partial(format_instructions = parser.get_format_instructions())

In [10]:
prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요.'), additional_kwargs

In [11]:
chain = prompt|model|parser
chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행

{'description': '지구 온난화는 기후 변화의 주요 원인으로, 극심한 날씨, 해수면 상승, 생태계 파괴 등을 초래합니다.',
 'hashtags': '#지구온난화 #기후변화'}